In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
##종웅님 
headers = {"X-Naver-Client-Id": "EmdlUXsc4CuKNm_kxYKS",
           "X-Naver-Client-Secret":"dSLoIEmMgy"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "모자", "param": [ "50000181"]},
          {"name": "벨트", "param": [ "50000180"]},
          {"name": "선글라스/안경테", "param": [ "50000183"]},
          {"name": "순금", "param": [ "50000432"]},
          {"name": "시계", "param": [ "50000425"]},
          {"name": "여성신발", "param": [ "50000173"]},
          {"name": "양말", "param": [ "50000166"]},
          {"name": "여행용가방/소품", "param": [ "50000178"]},
          {"name": "장갑", "param": [ "50000551"]},
          {"name": "주얼리", "param": [ "50000189"]},
          {"name": "지갑", "param": [ "50001471"]},
          {"name": "패션소품", "param": [ "50000567"]},
          {"name": "헤어액세서리", "param": [ "50000184"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "패션잡화", "param": [ "50000001"]},
      {"name": "남성신발", "param": [ "50000174"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "패션잡화")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "남성신발")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["패션잡화"], axis=1, inplace=True)
df_all_a

,남성신발,모자,벨트,선글라스/안경테,순금,시계,여성신발,양말,여행용가방/소품,장갑,주얼리,지갑,패션소품,헤어액세서리,유형,성별,연령
period,,,,,,,,,,,,,,,,,
2021-05-10,12.02174,2.44833,0.54212,1.51041,0.04076,0.02012,11.63245,0.96218,0.61878,0.02954,5.01534,0.92643,0.01057,0.76299,PC+MO,전체,전체
2021-05-11,11.66945,2.47859,0.51372,1.50514,0.03580,0.02205,11.48585,0.94191,0.69697,0.03402,4.88670,0.85049,0.00886,0.76663,PC+MO,전체,전체
2021-05-12,11.46025,2.61627,0.52574,1.44108,0.03470,0.01893,11.86613,0.93551,0.56297,0.03904,4.89193,0.78601,0.00822,0.77020,PC+MO,전체,전체
2021-05-13,10.91306,2.79239,0.47355,1.46638,0.02908,0.01762,11.97745,0.93313,0.60418,0.04460,4.51802,0.73853,0.00889,0.79353,PC+MO,전체,전체
2021-05-14,10.01901,2.55165,0.42827,1.39799,0.02860,0.01774,11.74289,0.85126,0.53751,0.03827,4.36706,0.74264,0.00620,0.73147,PC+MO,전체,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,6.20855,1.51461,0.29881,0.98156,0.04044,0.00483,10.29587,0.57851,1.18262,0.01307,3.83938,0.31975,0.00764,0.73063,MO,전체,전체
2023-06-30,5.28375,1.36258,0.26522,0.93482,0.03678,0.00433,8.29393,0.48174,1.00723,0.01106,3.62454,0.29212,0.00789,0.62227,MO,전체,전체
2023-07-01,6.67204,1.89104,0.32435,1.35277,0.04488,0.00501,9.50419,0.51789,1.29193,0.01788,4.43357,0.35768,0.00771,0.72239,MO,전체,전체


In [4]:
## 중분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "모자", "param": [ "50000181"]},
          {"name": "벨트", "param": [ "50000180"]},
          {"name": "선글라스/안경테", "param": [ "50000183"]},
          {"name": "순금", "param": [ "50000432"]},
          {"name": "시계", "param": [ "50000425"]},
          {"name": "여성신발", "param": [ "50000173"]},
          {"name": "양말", "param": [ "50000166"]},
          {"name": "여행용가방/소품", "param": [ "50000178"]},
          {"name": "장갑", "param": [ "50000551"]},
          {"name": "주얼리", "param": [ "50000189"]},
          {"name": "지갑", "param": [ "50001471"]},
          {"name": "패션소품", "param": [ "50000567"]},
          {"name": "헤어액세서리", "param": [ "50000184"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "패션잡화", "param": [ "50000001"]},
      {"name": "남성신발", "param": [ "50000174"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "패션잡화")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "남성신발")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["패션잡화"], axis=1, inplace=True)
df_all_b

,남성신발,모자,벨트,선글라스/안경테,순금,시계,여성신발,양말,여행용가방/소품,장갑,주얼리,지갑,패션소품,헤어액세서리,유형,성별,연령
period,,,,,,,,,,,,,,,,,
2021-05-10,7.99464,2.44600,0.41182,1.03378,0.04032,0.01195,16.78080,1.06667,0.59291,0.04021,6.38950,1.30428,0.01619,1.13662,PC+MO,여성,전체
2021-05-11,7.79660,2.46827,0.42463,1.07982,0.03358,0.01501,16.57859,1.01455,0.58932,0.04588,6.10003,1.18228,0.01369,1.15814,PC+MO,여성,전체
2021-05-12,7.76179,2.68186,0.42132,1.01885,0.03420,0.01285,17.22216,0.99746,0.55023,0.05500,6.18472,1.11414,0.01270,1.15900,PC+MO,여성,전체
2021-05-13,7.39246,3.04112,0.36223,1.05708,0.02828,0.01217,17.57169,0.97741,0.52180,0.06559,5.85993,1.02655,0.01478,1.21150,PC+MO,여성,전체
2021-05-14,6.74168,2.76990,0.33741,1.03053,0.02764,0.00994,17.26670,0.86268,0.45255,0.05442,5.62879,1.04677,0.00924,1.10034,PC+MO,여성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,4.86144,1.82331,0.27237,0.95660,0.04474,0.00228,16.92971,0.70225,1.46205,0.01793,5.88834,0.49060,0.01297,1.21988,MO,여성,전체
2023-06-30,4.11460,1.65476,0.24817,0.91488,0.03963,0.00271,13.57474,0.57218,1.21471,0.01606,5.56799,0.44947,0.01319,1.03253,MO,여성,전체
2023-07-01,5.37770,2.29034,0.30749,1.30990,0.05020,0.00311,15.54715,0.62032,1.58219,0.02571,6.81342,0.55201,0.01332,1.20548,MO,여성,전체


In [5]:
for i in devices:
    categories = [
          {"name": "모자", "param": [ "50000181"]},
          {"name": "벨트", "param": [ "50000180"]},
          {"name": "선글라스/안경테", "param": [ "50000183"]},
          {"name": "순금", "param": [ "50000432"]},
          {"name": "시계", "param": [ "50000425"]},
          {"name": "여성신발", "param": [ "50000173"]},
          {"name": "양말", "param": [ "50000166"]},
          {"name": "여행용가방/소품", "param": [ "50000178"]},
          {"name": "장갑", "param": [ "50000551"]},
          {"name": "주얼리", "param": [ "50000189"]},
          {"name": "지갑", "param": [ "50001471"]},
          {"name": "패션소품", "param": [ "50000567"]},
          {"name": "헤어액세서리", "param": [ "50000184"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "패션잡화", "param": [ "50000001"]},
      {"name": "남성신발", "param": [ "50000174"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "패션잡화")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "남성신발")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["패션잡화"], axis=1, inplace=True)
df_all_c

,남성신발,모자,벨트,선글라스/안경테,순금,시계,여성신발,양말,여행용가방/소품,장갑,주얼리,지갑,패션소품,헤어액세서리,유형,성별,연령
period,,,,,,,,,,,,,,,,,
2021-05-10,33.31996,3.79595,1.32090,2.91786,0.08268,0.05641,8.18329,1.41285,0.70407,0.02281,4.10042,0.69630,0.00438,0.37560,PC+MO,남성,전체
2021-05-11,32.05347,3.75816,1.20292,2.85480,0.07910,0.05647,7.67453,1.39562,0.67348,0.02024,3.93198,0.62446,0.00369,0.33806,PC+MO,남성,전체
2021-05-12,31.18791,3.81601,1.24567,2.70404,0.06744,0.04607,7.79696,1.43448,0.67048,0.02501,3.80204,0.54705,0.00257,0.35423,PC+MO,남성,전체
2021-05-13,29.37412,3.66751,1.19741,2.75776,0.06049,0.03980,7.36330,1.43066,0.63098,0.02294,3.56158,0.52310,0.00112,0.34113,PC+MO,남성,전체
2021-05-14,27.13858,3.39327,1.03919,2.44854,0.06055,0.04394,7.20910,1.29745,0.53852,0.02118,3.21537,0.51658,0.00231,0.33592,PC+MO,남성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,20.15462,2.87477,0.80592,2.58717,0.08887,0.01692,5.63689,0.94928,1.55943,0.01295,3.37350,0.27327,0.00363,0.32398,MO,남성,전체
2023-06-30,17.16047,2.55949,0.71060,2.42785,0.08490,0.01466,4.67130,0.84484,1.37112,0.01068,3.14421,0.23983,0.00411,0.28492,MO,남성,전체
2023-07-01,21.18222,3.60470,0.87685,3.59031,0.09511,0.01630,5.46215,0.93968,1.90466,0.01959,3.88669,0.29801,0.00191,0.32508,MO,남성,전체


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)

## 엑셀에 저장
df_all.to_excel("MCLS_10_all_fm_all.xlsx", sheet_name="click ratio")
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7031 entries, 2021-05-10 to 2023-07-03
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   남성신발      7031 non-null   float64
 1   모자        7031 non-null   float64
 2   벨트        7031 non-null   float64
 3   선글라스/안경테  7031 non-null   float64
 4   순금        7031 non-null   float64
 5   시계        7031 non-null   float64
 6   여성신발      7031 non-null   float64
 7   양말        7031 non-null   float64
 8   여행용가방/소품  7031 non-null   float64
 9   장갑        7031 non-null   float64
 10  주얼리       7031 non-null   float64
 11  지갑        7031 non-null   float64
 12  패션소품      7031 non-null   float64
 13  헤어액세서리    7031 non-null   float64
 14  유형        7031 non-null   object 
 15  성별        7031 non-null   object 
 16  연령        7031 non-null   object 
dtypes: float64(14), object(3)
memory usage: 988.7+ KB
